In [1]:
import hope
hope.config.optimize = True
hope.config.verbose = True
hope.config.keeptemp = True
import numba
import numpy as np
from util import perf_comp_data
from native_util import load
%load_ext cythonmagic
%load_ext version_information
%version_information numpy, Cython, numba, hope

The Cython magic has been move to the Cython package, hence 
`%load_ext cythonmagic` is deprecated; Please use `%load_ext Cython` instead.
You need Cython version >=0.21 to use the Cython magic


/Users/jakeret/Library/Python/2.7/lib/python/site-packages/IPython/core/formatters.py:832: FormatterWarning: JSON expects JSONable list/dict containers, not JSON strings
  FormatterWarning)


Software versions
Python 2.7.10 64bit [GCC 4.2.1 Compatible Apple LLVM 5.1 (clang-503.0.40)]
IPython 3.0.0
OS Darwin 13.4.0 x86_64 i386 64bit
numpy 1.8.1
Cython 0.20.2
numba 0.20.0
hope 0.4.0
Thu Jul 16 17:37:55 2015 CEST

# fibonacci

In [2]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)
hope_fib = hope.jit(fib)
numba_fib = numba.jit(fib, nopython=False)

native_fib_mod = load("fib")
native_fib = native_fib_mod.run 

n=20
assert fib(20) == 6765
assert hope_fib(20) == 6765
assert numba_fib(20) == 6765
assert native_fib(20) == 6765

running build_ext
building 'fib' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -fno-common -dynamic -pipe -Os -fwrapv -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

compile options: '-I/Users/jakeret/workspace/virtualenvs/hope_numba_benchmarks/lib/python2.7/site-packages/numpy/core/include -I/opt/local/Library/Frameworks/Python.framework/Versions/2.7/include/python2.7 -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11'
clang: ././src/fib.cpp
/usr/bin/clang++ -bundle -undefined dynamic_lookup -L/opt/local/lib -Wl,-headerpad_max_install_names -L/opt/local/lib/db48 ./src/fib.o -o ./fib.so



In [3]:
%%cython

cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef int cython_fib(int n):
    if n<2:
        return n
    return cython_fib(n-1)+cython_fib(n-2)

assert cython_fib(20) == 6765


ERROR: Cell magic `%%cython` not found.


In [4]:
%timeit fib(20)
%timeit hope_fib(20)
%timeit numba_fib(20)
%timeit cython_fib(20)
%timeit native_fib(20)

100 loops, best of 3: 2.52 ms per loop
10000 loops, best of 3: 39.5 µs per loop
100 loops, best of 3: 2.48 ms per loop


NameError: global name 'cython_fib' is not defined

In [ ]:
perf_comp_data(["fib", "hope_fib", "numba_fib", "cython_fib", "native_fib"],
               5*["n"])

# quicksort

In [ ]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    if False: return a
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                tmp = a[i]
                a[i] = a[j]
                a[j] = tmp
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

hope_qsort_kernel = hope.jit(qsort_kernel)
numba_qsort_kernel = numba.jit(qsort_kernel)

native_qsort_kernel_mod = load("qsort_kernel")
native_qsort_kernel = native_qsort_kernel_mod.run


In [ ]:
def numpy_qsort_kernel(a, lo, hi):
    np.sort(a)

In [ ]:
%%cython

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def cython_qsort_kernel(np.ndarray[np.double_t, ndim=1] a, int lo, int hi):
    cdef int i = lo
    cdef int j = hi
    cdef double pivot = 0
    cdef double tmp = 0.0
    if False: return a
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                tmp = a[i]
                a[i] = a[j]
                a[j] = tmp
                i += 1
                j -= 1
        if lo < j:
            cython_qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a



In [ ]:
lst = np.random.random(5000)


In [ ]:
psorted = qsort_kernel(lst.copy(), 0, len(lst)-1)
hsorted = hope_qsort_kernel(lst.copy(), 0, len(lst)-1)
#nsorted = numba_qsort_kernel(lst.copy(), 0, len(lst)-1)
csorted = cython_qsort_kernel(lst.copy(), 0, len(lst)-1)
nasorted = native_qsort_kernel(lst.copy(), 0, len(lst)-1)


In [ ]:
assert np.all(psorted[:-1] <= psorted[1:])
#assert np.all(hope_qsort_kernel[:-1] <= hope_qsort_kernel[1:])
#assert np.all(numba_qsort_kernel[:-1] <= numba_qsort_kernel[1:])
#assert np.all(cython_qsort_kernel[:-1] <= cython_qsort_kernel[1:])

%timeit qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit hope_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit numba_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit cython_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit native_qsort_kernel(lst.copy(), 0, len(lst)-1)
%timeit np.sort(lst.copy())

In [5]:
a = lst.copy()

lo = 0
hi = len(lst)-1

perf_comp_data(["hope_qsort_kernel", 
                "qsort_kernel", 
                #"numpy_qsort_kernel", 
                "cython_qsort_kernel", 
                "native_qsort_kernel"],
               5*["a, lo, hi"], rep=100, extra_setup="from __main__ import lst;a = lst.copy()")

NameError: name 'lst' is not defined

# pi sum

In [6]:
def pisum():
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            sum += 1.0/(k*k)
    return sum
def pisum_opt():
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            f += 1.
            sum += 1.0/(f*f)
    return sum

hope_pisum = hope.jit(pisum)
hope_pisum_opt = hope.jit(pisum_opt)

numba_pisum = numba.jit(pisum, nopython=True)
numba_pisum_opt = numba.jit(pisum_opt, nopython=True)

native_pisum_mod = load("pisum")
native_pisum = native_pisum_mod.run

native_pisum_opt_mod = load("pisum_opt")
native_pisum_opt = native_pisum_opt_mod.run


assert abs(pisum()-1.644834071848065) < 1e-6
assert abs(hope_pisum()-1.644834071848065) < 1e-6
assert abs(hope_pisum_opt()-1.644834071848065) < 1e-6
assert abs(numba_pisum()-1.644834071848065) < 1e-6
assert abs(native_pisum()-1.644834071848065) < 1e-6
assert abs(native_pisum_opt()-1.644834071848065) < 1e-6


running build_ext
building 'pisum' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -fno-common -dynamic -pipe -Os -fwrapv -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

compile options: '-I/Users/jakeret/workspace/virtualenvs/hope_numba_benchmarks/lib/python2.7/site-packages/numpy/core/include -I/opt/local/Library/Frameworks/Python.framework/Versions/2.7/include/python2.7 -c'
extra options: '-Wall -Wno-unused-variable -march=native -stdlib=libc++ -std=c++11'
clang: ././src/pisum.cpp
/usr/bin/clang++ -bundle -undefined dynamic_lookup -L/opt/local/lib -Wl,-headerpad_max_install_names -L/opt/local/lib/db48 ./src/pisum.o -o ./pisum.so

running build_ext
building 'pisum_opt' extension
C compiler: /usr/bin/clang -fno-strict-aliasing -fno-common -dynamic -pipe -Os -fwrapv -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

compile options: '-I/Users/jakeret/workspace/virtualenvs/hope_numba_benchmarks/lib/python2.7/site-packages/numpy/core/include -I/opt/local/Library/Frameworks/P

In [7]:
%load_ext cythonmagic


The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


In [8]:
%%cython

cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.locals(f=float)
def cython_pisum():
    cdef double sum = 0.0
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            sum += 1.0/(k*k)
    return sum

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.locals(f=float)
def cython_pisum_opt():
    cdef double sum = 0.0
    for j in range(1, 501):
        sum = 0.0
        f = 0.0
        for k in range(1, 10001):
            f += 1.
            sum += 1.0/(f*f)
    return sum


assert abs(cython_pisum()-1.644834071848065) < 1e-6
assert abs(cython_pisum_opt()-1.644834071848065) < 1e-6

ERROR: Cell magic `%%cython` not found.


In [9]:
%timeit pisum()
%timeit pisum_opt()
%timeit hope_pisum()
%timeit hope_pisum_opt()
%timeit numba_pisum()
%timeit numba_pisum_opt()
%timeit cython_pisum()
%timeit cython_pisum_opt()
%timeit native_pisum()
%timeit native_pisum_opt()

1 loops, best of 3: 519 ms per loop
1 loops, best of 3: 576 ms per loop
10 loops, best of 3: 43.9 ms per loop
10 loops, best of 3: 22.9 ms per loop
10 loops, best of 3: 42.7 ms per loop
The slowest run took 4.35 times longer than the fastest. This could mean that an intermediate result is being cached 
10 loops, best of 3: 23.2 ms per loop


NameError: global name 'cython_pisum' is not defined

In [10]:
perf_comp_data(["pisum", "pisum_opt", 
                "hope_pisum", "hope_pisum_opt", 
                "numba_pisum", "numba_pisum_opt", 
                #"cython_pisum", 
                #"cython_pisum_opt",
                "native_pisum", "native_pisum_opt",], 
               None, rep=100)

function: native_pisum_opt    , av. time sec:   0.02146053, min. time sec:   0.02097297, relative:       1.0
function: hope_pisum_opt      , av. time sec:   0.02149999, min. time sec:   0.02132988, relative:       1.0
function: numba_pisum_opt     , av. time sec:   0.02157497, min. time sec:   0.02127409, relative:       1.0
function: numba_pisum         , av. time sec:   0.03994894, min. time sec:   0.03944707, relative:       1.9
function: hope_pisum          , av. time sec:   0.04127896, min. time sec:   0.04057693, relative:       1.9
function: native_pisum        , av. time sec:   0.04172409, min. time sec:   0.04042220, relative:       1.9
function: pisum               , av. time sec:   0.51674390, min. time sec:   0.48155212, relative:      24.1
function: pisum_opt           , av. time sec:   0.60113752, min. time sec:   0.57436490, relative:      28.0
